# ☀️ SolarGrid DZ — 48-Hour Solar Production Forecast Analysis
**Algeria Solar Energy Management System**

This notebook covers:
1. Data exploration & EDA
2. Feature engineering (solar position, weather lags)
3. Gradient Boosting model training with weather correction
4. Model evaluation (MAE, RMSE, MAPE)
5. 48-hour ahead forecast generation
6. Visualization vs actual production

In [ ]:
## 1. Import Required Libraries
import sys
import os
sys.path.insert(0, os.path.abspath(".."))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime, timedelta

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb

plt.style.use("dark_background")
sns.set_theme(style="darkgrid", palette="muted")
plt.rcParams.update({"figure.dpi": 120, "figure.figsize": (14, 5)})

print("Libraries loaded ✓")
print(f"NumPy {np.__version__} | Pandas {pd.__version__}")

In [ ]:
## 2. Load and Explore Solar Production Data
from ml.solar_forecast import generate_synthetic_data

# Generate 2 years of hourly solar + weather data for Algiers (500 kW rooftop array)
df = generate_synthetic_data(start="2023-01-01", end="2024-12-31", capacity_kw=500.0, location="algiers")
print(f"Dataset: {len(df):,} hourly records ({df.index.min().date()} → {df.index.max().date()})")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

In [ ]:
## EDA — Descriptive Statistics & Seasonality
print(df.describe().round(2))

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Monthly average production
monthly = df["production_kw"].resample("ME").mean()
axes[0, 0].bar(monthly.index.strftime("%b %y"), monthly.values, color="orange", alpha=0.8)
axes[0, 0].set_title("Monthly Average Solar Production (kW)")
axes[0, 0].set_ylabel("kW"); axes[0, 0].tick_params(axis="x", rotation=45)

# Hourly profile by season
df["hour"] = df.index.hour
df["month"] = df.index.month
df["season"] = pd.cut(df["month"], bins=[0,3,6,9,12], labels=["Winter","Spring","Summer","Autumn"])
seasonal = df.groupby(["season","hour"])["production_kw"].mean().unstack(0)
seasonal.plot(ax=axes[0, 1], linewidth=2, colormap="plasma")
axes[0, 1].set_title("Hourly Production Profile by Season"); axes[0, 1].set_xlabel("Hour of Day")

# Cloud cover distribution
axes[1, 0].hist(df["cloud_cover"], bins=40, color="#3b82f6", alpha=0.8, edgecolor="none")
axes[1, 0].set_title("Cloud Cover Distribution (%)"); axes[1, 0].set_xlabel("%")

# Production vs GHI scatter
sample = df[df["ghi_wm2"] > 0].sample(3000, random_state=42)
axes[1, 1].scatter(sample["ghi_wm2"], sample["production_kw"], s=4, alpha=0.4, c="#f59e0b")
axes[1, 1].set_title("Production vs GHI Irradiance")
axes[1, 1].set_xlabel("GHI (W/m²)"); axes[1, 1].set_ylabel("Production (kW)")

plt.tight_layout(); plt.show()

In [ ]:
## 3. Feature Engineering
import math

def solar_elevation_demo(lat: float, lon: float, dt_index: pd.DatetimeIndex) -> np.ndarray:
    """Vectorised solar elevation angle (degrees)."""
    lat_r = math.radians(lat)
    doy = dt_index.day_of_year
    hour_angle = np.radians((dt_index.hour + dt_index.minute / 60 - 12) * 15)
    declination = np.radians(23.45 * np.sin(np.radians(360 / 365 * (doy - 81))))
    sin_elev = np.sin(lat_r) * np.sin(declination) + np.cos(lat_r) * np.cos(declination) * np.cos(hour_angle)
    return np.degrees(np.arcsin(np.clip(sin_elev, -1, 1)))

# Engineer features
df_feat = df.copy()
df_feat["hour_sin"] = np.sin(2 * np.pi * df_feat.index.hour / 24)
df_feat["hour_cos"] = np.cos(2 * np.pi * df_feat.index.hour / 24)
df_feat["doy_sin"]  = np.sin(2 * np.pi * df_feat.index.day_of_year / 365)
df_feat["doy_cos"]  = np.cos(2 * np.pi * df_feat.index.day_of_year / 365)
df_feat["solar_elevation"] = solar_elevation_demo(36.74, 3.06, df_feat.index)
df_feat["cloud_factor"]    = 1 - df_feat["cloud_cover"] / 100
df_feat["lag_1h"]  = df_feat["production_kw"].shift(1)
df_feat["lag_24h"] = df_feat["production_kw"].shift(24)
df_feat["roll_3h"] = df_feat["production_kw"].rolling(3).mean()
df_feat.dropna(inplace=True)

FEATURE_COLS = ["hour_sin","hour_cos","doy_sin","doy_cos","solar_elevation",
                "temperature_c","cloud_cover","cloud_factor","ghi_wm2",
                "wind_speed_ms","lag_1h","lag_24h","roll_3h"]

print("Feature matrix shape:", df_feat[FEATURE_COLS].shape)

# Correlation heatmap
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
corr = df_feat[FEATURE_COLS + ["production_kw"]].corr()
mask = np.triu(np.ones_like(corr))
sns.heatmap(corr, mask=mask, ax=axes[0], cmap="coolwarm", center=0,
            annot=False, linewidths=0.4, cbar_kws={"shrink": 0.8})
axes[0].set_title("Feature Correlation Matrix")

# Solar elevation curve — summer vs winter day
summer_day = df_feat[df_feat.index.date == df_feat.index[len(df_feat)//2].date()].head(24)
winter_day = df_feat[df_feat.index.date == df_feat.index[0].date()].head(24)
axes[1].plot(range(len(summer_day)), summer_day["solar_elevation"].values, label="Summer", color="#f59e0b")
axes[1].plot(range(len(winter_day)), winter_day["solar_elevation"].values, label="Winter", color="#3b82f6", linestyle="--")
axes[1].axhline(0, color="gray", linewidth=0.8)
axes[1].set_title("Solar Elevation Angle — Summer vs Winter")
axes[1].set_xlabel("Hour of Day"); axes[1].set_ylabel("Degrees"); axes[1].legend()
plt.tight_layout(); plt.show()

In [ ]:
## 4. Train Gradient Boosting & XGBoost Models (Time-Series Split)
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error

X = df_feat[FEATURE_COLS].values
y = df_feat["production_kw"].values

# Time-aware train / val split (last 20% as validation)
split_idx = int(len(X) * 0.80)
X_train, X_val = X[:split_idx], X[split_idx:]
y_train, y_val = y[:split_idx], y[split_idx:]
print(f"Train samples: {len(X_train):,}  |  Val samples: {len(X_val):,}")

# Gradient Boosting
gbr_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("gbr", GradientBoostingRegressor(
        n_estimators=300, learning_rate=0.05, max_depth=5,
        subsample=0.8, min_samples_leaf=10, random_state=42, verbose=0
    ))
])
print("Training GBR…")
gbr_pipe.fit(X_train, y_train)

# XGBoost
try:
    from xgboost import XGBRegressor
    xgb_model = Pipeline([
        ("scaler", StandardScaler()),
        ("xgb", XGBRegressor(
            n_estimators=400, learning_rate=0.05, max_depth=6,
            subsample=0.8, colsample_bytree=0.8, random_state=42,
            verbosity=0, n_jobs=-1
        ))
    ])
    print("Training XGBoost…")
    xgb_model.fit(X_train, y_train)
    has_xgb = True
except ImportError:
    print("XGBoost not available; skipping.")
    has_xgb = False

print("Training complete ✓")

In [ ]:
## 5. Model Evaluation — MAE / RMSE / MAPE + Feature Importance
def evaluate(name, model, X_val, y_val, capacity_kw=500.0):
    preds = np.clip(model.predict(X_val), 0, capacity_kw)
    mae  = mean_absolute_error(y_val, preds)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    # MAPE over daylight hours only
    mask = y_val > capacity_kw * 0.02
    mape = np.mean(np.abs((y_val[mask] - preds[mask]) / (y_val[mask] + 1e-6))) * 100
    norm_mae = mae / capacity_kw * 100
    print(f"{'='*50}\n{name}")
    print(f"  MAE  : {mae:6.2f} kW  ({norm_mae:.2f}% of capacity)")
    print(f"  RMSE : {rmse:6.2f} kW")
    print(f"  MAPE : {mape:.2f}%  (daylight hours)")
    return preds, mae, rmse, mape

gbr_preds, *gbr_scores = evaluate("Gradient Boosting Regressor", gbr_pipe, X_val, y_val)
if has_xgb:
    xgb_preds, *xgb_scores = evaluate("XGBoost Regressor", xgb_model, X_val, y_val)

# Plot predictions vs actuals (first 7 days of validation)
fig, axes = plt.subplots(2, 1, figsize=(16, 9), sharex=True)
n_plot = min(168, len(y_val))  # 7 days
t = np.arange(n_plot)

axes[0].fill_between(t, 0, y_val[:n_plot], alpha=0.25, color="#f59e0b", label="Actual")
axes[0].plot(t, y_val[:n_plot], color="#f59e0b", linewidth=1.2, label="_nolegend_")
axes[0].plot(t, gbr_preds[:n_plot], color="#3b82f6", linewidth=1.4, label="GBR Forecast")
if has_xgb:
    axes[0].plot(t, xgb_preds[:n_plot], color="#10b981", linewidth=1.4, linestyle="--", label="XGB Forecast")
axes[0].set_title("Forecast vs Actual — Validation Set (First 7 Days)")
axes[0].set_ylabel("Production (kW)"); axes[0].legend(); axes[0].grid(alpha=0.3)

# Residuals
axes[1].bar(t, y_val[:n_plot] - gbr_preds[:n_plot], color="#ef4444", alpha=0.6, label="GBR Residuals")
axes[1].axhline(0, color="gray", linewidth=0.8)
axes[1].set_title("GBR Residuals (Actual − Predicted)")
axes[1].set_xlabel("Hour"); axes[1].set_ylabel("kW"); axes[1].grid(alpha=0.3)
plt.tight_layout(); plt.show()

# Feature importance
fig, ax = plt.subplots(figsize=(10, 5))
importances = gbr_pipe.named_steps["gbr"].feature_importances_
order = np.argsort(importances)[::-1]
ax.barh([FEATURE_COLS[i] for i in order], importances[order], color="#f59e0b", alpha=0.85)
ax.set_title("GBR Feature Importances"); ax.set_xlabel("Importance")
ax.invert_yaxis(); plt.tight_layout(); plt.show()

In [ ]:
## 6. Generate 48-Hour Probabilistic Forecast (P10 / P50 / P90)
import requests

HORIZON = 48
LAT, LON = 36.74, 3.06
CAPACITY_KW = 500.0

def fetch_nwp_forecast(lat: float, lon: float, hours: int = 48) -> pd.DataFrame:
    """Fetch free NWP forecast from Open-Meteo (no API key required)."""
    url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat}&longitude={lon}"
        f"&hourly=temperature_2m,cloudcover,windspeed_10m,shortwave_radiation"
        f"&timezone=Africa/Algiers&forecast_days=3"
    )
    try:
        resp = requests.get(url, timeout=8)
        resp.raise_for_status()
        data = resp.json()["hourly"]
        df_wx = pd.DataFrame({
            "temperature_c": data["temperature_2m"],
            "cloud_cover":   data["cloudcover"],
            "wind_speed_ms": data["windspeed_10m"],
            "ghi_wm2":       data["shortwave_radiation"],
        }, index=pd.to_datetime(data["time"]))
        print(f"Open-Meteo: fetched {len(df_wx)} NWP hours ✓")
        return df_wx.iloc[:hours]
    except Exception as e:
        print(f"Open-Meteo unavailable ({e}). Using synthetic NWP.")
        return None

df_wx = fetch_nwp_forecast(LAT, LON, HORIZON)

# Build feature rows for the next 48 hours
now = pd.Timestamp.now(tz="Africa/Algiers").floor("h").tz_localize(None)
future_idx = pd.date_range(now, periods=HORIZON, freq="h")

if df_wx is not None and len(df_wx) >= HORIZON:
    wx = df_wx.iloc[:HORIZON].copy()
    wx.index = future_idx
else:
    # Synthetic NWP fallback — clear summer day profile
    hours_arr = future_idx.hour
    wx = pd.DataFrame({
        "temperature_c": 28 + 5 * np.sin(np.pi * hours_arr / 24),
        "cloud_cover":   15 * np.abs(np.sin(np.pi * hours_arr / 12)),
        "wind_speed_ms": 3 + 1.5 * np.random.randn(HORIZON),
        "ghi_wm2":       np.clip(850 * np.sin(np.pi * np.clip(hours_arr - 5, 0, 14) / 14) + np.random.randn(HORIZON) * 20, 0, None),
    }, index=future_idx)

wx["hour_sin"] = np.sin(2 * np.pi * wx.index.hour / 24)
wx["hour_cos"] = np.cos(2 * np.pi * wx.index.hour / 24)
wx["doy_sin"]  = np.sin(2 * np.pi * wx.index.day_of_year / 365)
wx["doy_cos"]  = np.cos(2 * np.pi * wx.index.day_of_year / 365)
wx["solar_elevation"] = solar_elevation_demo(LAT, LON, wx.index)
wx["cloud_factor"]    = 1 - wx["cloud_cover"] / 100
# Lag features — use last known actuals
last_prod = df_feat["production_kw"].iloc[-24:].values
wx["lag_1h"]  = np.concatenate([[last_prod[-1]], np.full(HORIZON - 1, np.nan)])
wx["lag_24h"] = np.concatenate([last_prod[-HORIZON:], np.full(max(0, HORIZON - 24), np.nan)])[:HORIZON]
wx["roll_3h"] = pd.Series(wx["lag_1h"]).fillna(method="ffill").rolling(3, min_periods=1).mean().values
wx.fillna(method="ffill", inplace=True)

X_fc = wx[FEATURE_COLS].values

# P50 (best estimate)
p50 = np.clip(gbr_pipe.predict(X_fc), 0, CAPACITY_KW)

# P10/P90 via additive noise scaled to residual std
resid_std = np.std(y_val - gbr_preds)
p10 = np.clip(p50 - 1.28 * resid_std, 0, CAPACITY_KW)
p90 = np.clip(p50 + 1.28 * resid_std, 0, CAPACITY_KW)

forecast_df = pd.DataFrame({"p10": p10, "p50": p50, "p90": p90}, index=future_idx)
print(forecast_df.head(8).round(1))
print(f"\nPeak P50 forecast: {p50.max():.1f} kW at hour {p50.argmax()}")

In [ ]:
## 7. Visualise 48-Hour Probabilistic Forecast
fig = plt.figure(figsize=(16, 12))
gs  = fig.add_gridspec(3, 2, hspace=0.4, wspace=0.3)

ax1 = fig.add_subplot(gs[0, :])   # full-width forecast ribbon
ax2 = fig.add_subplot(gs[1, 0])   # hourly uncertainty band
ax3 = fig.add_subplot(gs[1, 1])   # energy balance bar
ax4 = fig.add_subplot(gs[2, :])   # cumulative energy + curtailment zone

t = forecast_df.index

# — Forecast Ribbon —
ax1.fill_between(t, forecast_df.p10, forecast_df.p90, alpha=0.25, color="#f59e0b", label="P10–P90 band")
ax1.plot(t, forecast_df.p50, color="#f59e0b", linewidth=2.5, label="P50 forecast")
ax1.plot(t, forecast_df.p10, color="#f59e0b", linewidth=0.8, linestyle="--", label="P10 / P90")
ax1.plot(t, forecast_df.p90, color="#f59e0b", linewidth=0.8, linestyle="--")
ax1.axhline(CAPACITY_KW * 0.9, color="#ef4444", linewidth=1, linestyle=":", label="90% curtailment threshold")
ax1.set_title("48-Hour Solar Production Forecast — Algiers District α")
ax1.set_ylabel("Production (kW)"); ax1.legend(loc="upper right"); ax1.grid(alpha=0.3)
ax1.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter("%d %b %Hh"))
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=30, ha="right")

# — Uncertainty band width —
uncertainty = forecast_df.p90 - forecast_df.p10
ax2.bar(range(len(t)), uncertainty, color="#8b5cf6", alpha=0.75)
ax2.set_title("Forecast Uncertainty (P90−P10)")
ax2.set_xlabel("Hours ahead"); ax2.set_ylabel("kW"); ax2.grid(alpha=0.3)

# — Hourly energy balance (production - flat 300 kW demand baseline) —
demand_baseline = 300
net = forecast_df.p50 - demand_baseline
colors_net = ["#10b981" if v >= 0 else "#ef4444" for v in net]
ax3.bar(range(len(t)), net, color=colors_net, alpha=0.82)
ax3.axhline(0, color="gray", linewidth=0.8)
ax3.set_title("Net Energy (Production − Demand, 300 kW baseline)")
ax3.set_xlabel("Hours ahead"); ax3.set_ylabel("kW"); ax3.grid(alpha=0.3)

# — Cumulative production + curtailment zones —
cum_energy   = np.cumsum(forecast_df.p50) / 1000   # MWh
curtailment  = np.clip(forecast_df.p50 - CAPACITY_KW * 0.9, 0, None)
cum_curtail  = np.cumsum(curtailment) / 1000

ax4.fill_between(range(len(t)), cum_curtail, cum_energy, alpha=0.6, color="#f59e0b", label="Usable energy (MWh)")
ax4.fill_between(range(len(t)), 0, cum_curtail, alpha=0.5, color="#ef4444", label="Curtailed energy (MWh)")
ax4.plot(range(len(t)), cum_energy, color="#f59e0b", linewidth=2)
ax4.set_title("Cumulative Energy — 48-Hour Horizon")
ax4.set_xlabel("Hours ahead"); ax4.set_ylabel("Cumulative MWh"); ax4.legend(); ax4.grid(alpha=0.3)

plt.suptitle("SolarGrid DZ — Solar Forecast Analysis Notebook", fontsize=14, fontweight="bold", y=1.01)
plt.savefig("notebooks/solar_forecast_analysis.png", dpi=150, bbox_inches="tight")
plt.show()

# Summary stats
total_mwh      = forecast_df.p50.sum() / 1000
curtailed_mwh  = curtailment.sum() / 1000
peak_kw        = forecast_df.p50.max()
peak_hour      = t[forecast_df.p50.argmax()].strftime("%d %b %H:%M")
co2_saved_tons = total_mwh * 0.420   # ENTSO-E Algeria emission factor

print(f"\n{'='*55}")
print(f"  48-Hour Forecast Summary")
print(f"{'='*55}")
print(f"  Total Expected Energy : {total_mwh:.2f} MWh")
print(f"  Peak Production       : {peak_kw:.1f} kW  at {peak_hour}")
print(f"  Est. Curtailment      : {curtailed_mwh:.3f} MWh")
print(f"  CO₂ Avoided           : {co2_saved_tons:.2f} tonnes")
print(f"{'='*55}")

## Summary

| Step | Artifact | Notes |
|------|----------|-------|
| Data | 2-year synthetic dataset (17,520 rows) | Algiers coordinates, 500 kW capacity |
| Features | 13 engineered features | Solar elevation, cyclical encoding, NWP lags |
| Model | GradientBoostingRegressor + XGBoost | Time-series split (80/20) |
| Target accuracy | < 5% normalised MAE | On par with literature for GHI-based forecasts |
| Forecast | 48-hour P10/P50/P90 | Open-Meteo NWP or synthetic fallback |
| Output | `notebooks/solar_forecast_analysis.png` | 4-panel summary figure |

### Next Steps
- Pull real SONELGAZ smart-meter data via the Kafka pipeline (`backend/data_pipeline/kafka_consumer.py`)
- Serve live forecasts through `GET /api/v1/forecasts/solar/{id}?horizon=48`
- Schedule weekly retraining with Celery Beat (`backend/tasks.py → retrain_solar_forecast`)
- Run `python ml/train_all.py` to persist models to `ml/saved_models/`